## 🍎 Classificador de Frutas com Rede Neural Convolucional (CNN)

Este notebook apresenta um sistema completo de **classificação de frutas** utilizando **Deep Learning**, baseado em uma **Rede Neural Convolucional (CNN)** desenvolvida com **TensorFlow/Keras**.

O objetivo é treinar um modelo capaz de identificar diferentes tipos de frutas a partir de imagens, utilizando um pipeline robusto de preparação dos dados, construção do modelo, monitoramento de métricas e salvamento de resultados.

---

## 📌 Funcionalidades do Notebook

### **1. Pré-processamento das imagens**
- Redimensionamento automático para o tamanho definido
- Normalização dos pixels para acelerar o treinamento
- *Data augmentation* para reduzir overfitting e aumentar a robustez:
  - rotações
  - zoom
  - flips horizontais
  - variações de brilho

---

### **2. Construção do Modelo (CNN)**
A arquitetura inclui:
- Múltiplas camadas **Convolution2D**
- **MaxPooling2D** para redução de dimensionalidade
- **Batch Normalization** para estabilizar o treino
- **Dropout** para diminuir overfitting
- Otimizador **Adam**
- Função de perda **categorical_crossentropy**

---

### **3. Treinamento e Validação**
- Separação dos dados em **treino**, **validação** e **teste**
- Callbacks profissionais:
  - **EarlyStopping** (interrompe quando não melhora)
  - **ModelCheckpoint** (salva o melhor modelo)
- Métricas acompanhadas:
  - `accuracy`, `val_accuracy`
  - `loss`, `val_loss`
- Pipeline de dados otimizado com `tf.data.AUTOTUNE` para alto desempenho

---

### **4. Monitoramento com MLflow**
O MLflow registra automaticamente:
- Parâmetros do experimento
- Hiperparâmetros do modelo
- Curvas de treino e validação
- Arquitetura e pesos do modelo (artefatos)
- Melhor versão do modelo salvo

Isso permite comparar execuções e reproduzir treinamentos.

---

### **5. Predição em Novas Imagens**
- Carregamento e pré-processamento automático
- Inferência pelo modelo treinado
- Exibição da imagem com a **classe prevista**
- Probabilidade da previsão exibida junto ao resultado

---

## 🛠️ Tecnologias Utilizadas
- **Python**
- **TensorFlow / Keras**
- **NumPy**
- **OpenCV**
- **Matplotlib**
- **MLflow**
- **GPU com Memory Growth ativado** para evitar problemas de VRAM

---

### Download Automático do Dataset (Kaggle)

Nesta etapa, o notebook realiza o **download automático do dataset** diretamente do Kaggle usando a biblioteca `kagglehub`.  
Para isso, é criado o arquivo `kaggle.json` contendo as credenciais da conta, permitindo o acesso à API do Kaggle.

**O que o código faz:**
1. Instala a biblioteca `kagglehub` (caso não esteja instalada).
2. Cria a pasta `~/.kaggle` caso ela ainda não exista.
3. Gera o arquivo `kaggle.json` com seu nome de usuário e chave de API.
4. Ajusta permissões do arquivo para que o Kaggle aceite o acesso.
5. Baixa automaticamente o dataset **"Fruit Classification (10 classes)"**.
6. Exibe o caminho da pasta onde o dataset foi salvo.

Após esta etapa, o dataset está disponível localmente e pode ser carregado para pré-processamento e treinamento do modelo.

In [ ]:
!pip install kagglehub --quiet
import os, json

kaggle_config = {
    "username": "SEU_USUARIO",
    "key": "SUA_SENHA"}

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_config, f)

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

import kagglehub

dataset_handle = "karimabdulnabi/fruit-classification10-class"
path = kagglehub.dataset_download(dataset_handle, force_download=True)

print("✅ Dataset baixado em:", path)

### Padronização dos nomes das classes

Como o dataset pode conter pastas com nomes inconsistentes (ex.: `stawberries`, `pinenapple`), esta etapa garante que **todas as classes tenham nomes padronizados e corrigidos** antes do treinamento.

O código faz:

1. **Lê automaticamente** os nomes das pastas dentro dos diretórios `train` e `test`;
2. Aplica um dicionário de correções (`standard_names`), corrigindo erros de digitação e padronizando capitalização;
3. Retorna a lista final ordenada das classes já corrigidas.

Isso evita que classes diferentes sejam tratadas como distintas apenas devido a erros de escrita no nome da pasta.


In [ ]:
import os

# Dicionário de padronização
standard_names = {
    'apple': 'Apple',
    'banana': 'Banana',
    'avocado': 'Avocado',
    'cherry': 'Cherry',
    'kiwi': 'Kiwi',
    'mango': 'Mango',
    'orange': 'Orange',
    'pinenapple': 'Pineapple',
    'pineapple': 'Pineapple',
    'strawberries': 'Strawberries',
    'stawberries': 'Strawberries',
    'watermelon': 'Watermelon'
}

def standardize_class_names(class_list):
    return sorted([standard_names.get(c.lower(), c) for c in class_list])

# ⚙️ Lê as classes diretamente das pastas
train_dir = '/home/wallingson12/Github/Data science and ML/Classificador de frutas/train'
validation_dir = '/home/wallingson12/Github/Data science and ML/Classificador de frutas/test'

train_classes_raw = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
val_classes_raw = [d for d in os.listdir(validation_dir) if os.path.isdir(os.path.join(validation_dir, d))]

# ✅ Padroniza
train_std = standardize_class_names(train_classes_raw)
val_std   = standardize_class_names(val_classes_raw)

### Verificação das classes após padronização

Após corrigir e padronizar os nomes das pastas, é importante confirmar se o conjunto de **treino** e o de **validação** possuem exatamente as mesmas classes.

O código:

- Exibe no console as classes padronizadas de cada conjunto;
- Compara ambas as listas para garantir que não existe uma fruta presente somente em um dos conjuntos;
- Caso haja discrepâncias, ele indica quais classes estão faltando em cada conjunto.

Isso evita erros futuros, como a rede ser treinada com uma classe que não aparece no conjunto de validação, o que afetaria as métricas de avaliação e o processo de generalização.

In [ ]:
print("▶ Classes padronizadas (treino):", train_std)
print("▶ Classes padronizadas (validação):", val_std)

# Verifica se há classes faltando em algum conjunto
missing_in_val = set(train_std) - set(val_std)
missing_in_train = set(val_std) - set(train_std)

if not missing_in_val and not missing_in_train:
    print("\nPerfeito! Treino e validação agora possuem as MESMAS classes.")
else:
    print("\n⚠ Diferenças encontradas:")
    if missing_in_val:
        print(" - Presentes no treino mas ausentes na validação:", missing_in_val)
    if missing_in_train:
        print(" - Presentes na validação mas ausentes no treino:", missing_in_train)

## Data Augmentation e Pré-processamento

In [3]:
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf

def build_model(image_size, num_classes):
    l2 = regularizers.l2(1e-4)

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.10),
        layers.RandomContrast(0.10),
    ])

    model = models.Sequential([
        layers.Input(shape=(image_size[0], image_size[1], 3)),
        data_augmentation,
        layers.Rescaling(1./255),

        # BLOCO 1
        layers.Conv2D(64, 3, padding='same', kernel_initializer="he_normal", kernel_regularizer=l2),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, 3, padding='same', kernel_initializer="he_normal", kernel_regularizer=l2),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.15),

        # BLOCO 2
        layers.Conv2D(128, 3, padding='same', kernel_initializer="he_normal", kernel_regularizer=l2),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(128, 3, padding='same', kernel_initializer="he_normal", kernel_regularizer=l2),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.20),

        # BLOCO 3
        layers.Conv2D(256, 3, padding='same', kernel_initializer="he_normal", kernel_regularizer=l2),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(256, 3, padding='same', kernel_initializer="he_normal", kernel_regularizer=l2),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.GlobalAveragePooling2D(),

        # Densa
        layers.Dense(512, kernel_initializer="he_normal", kernel_regularizer=l2),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dropout(0.30),

        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

## Treinamento

In [4]:
import os
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import pandas as pd

def train_model(model, train_dir, validation_dir, image_size=(128,128), batch_size=16, epochs=10, save_dir="./histories"):
    os.makedirs(save_dir, exist_ok=True)

    os.environ["TF_DETERMINISTIC_OPS"] = "0"
    AUTOTUNE = tf.data.AUTOTUNE

    print("Carregando datasets...")
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        train_dir,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=True
    )

    val_dataset = tf.keras.utils.image_dataset_from_directory(
        validation_dir,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='categorical',
        shuffle=False
    )

    # otimização p/ GPU
    train_dataset = train_dataset.cache().prefetch(AUTOTUNE)
    val_dataset = val_dataset.cache().prefetch(AUTOTUNE)

    # Callbacks
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=1e-6
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    )

    checkpoint = ModelCheckpoint(
        filepath=os.path.join(save_dir, 'best_model.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )

    print("Iniciando treino...")
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=val_dataset,
        callbacks=[reduce_lr, early_stop, checkpoint]
    )

    # Salvar histórico
    history_file = os.path.join(save_dir, 'history.csv')
    pd.DataFrame(history.history).to_csv(history_file, index=False)
    print(f"Histórico salvo em: {history_file}")
    print(f"Modelo salvo em: {os.path.join(save_dir, 'best_model.keras')}")

    return history

In [ ]:
import os


# Configurações da imagem
image_size = (128, 128)
batch_size = 64
epochs = 40

# Detectando automaticamente as classes pelas pastas do treino
class_names = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
num_classes = len(class_names)

print(f"Classes detectadas: {class_names}")
print(f"Número de classes: {num_classes}")

# Criando o modelo
model = build_model(image_size=image_size, num_classes=num_classes)

# Treinando o modelo
history = train_model(
    model,
    train_dir,
    validation_dir,
    image_size,
    batch_size=batch_size,
    epochs=epochs,
    save_dir="./histories"
)

## Acurácia de validação máxima: 0.4790 (47.90%)

# Geração de Variações de Imagens (Data Augmentation Offline)

Este bloco de código implementa um procedimento de *data augmentation offline*, destinado a ampliar o conjunto de dados utilizado no treinamento de modelos de visão computacional. A técnica consiste em gerar múltiplas variações artificiais de cada imagem original, com o objetivo de melhorar a capacidade de generalização do modelo e mitigar problemas de sobreajuste (*overfitting*).

As variações aplicadas incluem:

- Rotações em ângulos fixos e aleatórios;  
- Espelhamento horizontal e vertical;  
- Alterações aleatórias de brilho;  
- Alterações aleatórias de contraste;  
- Inserção de ruído na imagem;  
- Redimensionamento para padronização.

As imagens geradas são salvas em um diretório específico, mantendo-se a estrutura de categorias existente no diretório de origem. Esse processo permite aumentar a diversidade do conjunto de treinamento e aproximar o modelo de situações reais de variação visual, contribuindo para um desempenho mais consistente durante a inferência.

In [1]:
import os
from PIL import Image, ImageEnhance
import numpy as np
import random

# --------------------------------------------------------------------
# FUNÇÃO: Criar variações de uma única imagem
# --------------------------------------------------------------------
def criar_variacoes(imagem_path, output_dir):
    img = Image.open(imagem_path).convert('RGB')
    W, H = img.size
    variacoes = []

    # Rotações
    for angulo in [90, 180, 270, random.randint(10, 50)]:
        variacoes.append(img.rotate(angulo).resize((W,H)))

    # Espelhamentos
    variacoes.append(img.transpose(Image.FLIP_LEFT_RIGHT))
    variacoes.append(img.transpose(Image.FLIP_TOP_BOTTOM))

    # Brilho
    enhancer_b = ImageEnhance.Brightness(img)
    for _ in range(3):
        variacoes.append(enhancer_b.enhance(random.uniform(0.6, 1.4)))

    # Contraste
    enhancer_c = ImageEnhance.Contrast(img)
    for _ in range(3):
        variacoes.append(enhancer_c.enhance(random.uniform(0.6, 1.4)))

    # Ruído
    arr = np.array(img)
    for _ in range(3):
        noise = np.random.randint(-20, 20, arr.shape, dtype='int16')
        noisy = np.clip(arr.astype('int16') + noise, 0, 255).astype('uint8')
        variacoes.append(Image.fromarray(noisy))

    # Salvar
    base = os.path.splitext(os.path.basename(imagem_path))[0]
    for i, v in enumerate(variacoes):
        out_path = os.path.join(output_dir, f"{base}_var{i}.jpg")
        v.save(out_path, quality=95)
        # print(f"[OK] {out_path}")

# --------------------------------------------------------------------
# FUNÇÃO: Criar variações para todas as pastas
# --------------------------------------------------------------------
def gerar_variacoes_em_diretorio(diretorio_origem, diretorio_saida):
    os.makedirs(diretorio_saida, exist_ok=True)

    for categoria in os.listdir(diretorio_origem):
        origem_cat = os.path.join(diretorio_origem, categoria)

        if os.path.isdir(origem_cat):
            saida_cat = os.path.join(diretorio_saida, categoria)
            os.makedirs(saida_cat, exist_ok=True)

            print(f"\n📂 Categoria: {categoria}")
            for arquivo in os.listdir(origem_cat):
                caminho = os.path.join(origem_cat, arquivo)

                if arquivo.lower().endswith(('.jpg', '.jpeg', '.png')):
                    criar_variacoes(caminho, saida_cat)

            total = len(os.listdir(saida_cat))
            print(f"✔ Total gerado em '{categoria}': {total}")

# --------------------------------------------------------------------
# EXECUTAR AQUI
# --------------------------------------------------------------------
train_dir = "/home/wallingson12/Github/Data science and ML/Classificador de frutas/train"
output_dir = "/home/wallingson12/Github/Data science and ML/Classificador de frutas/train_variacoes"

print("=== GERANDO VARIAÇÕES ===")
gerar_variacoes_em_diretorio(train_dir, output_dir)
print("\n🎉 PRONTO! TODAS AS VARIAÇÕES FORAM GERADAS.")

=== GERANDO VARIAÇÕES ===

📂 Categoria: pinenapple
✔ Total gerado em 'pinenapple': 3450

📂 Categoria: avocado
✔ Total gerado em 'avocado': 3450

📂 Categoria: cherry
✔ Total gerado em 'cherry': 3450

📂 Categoria: Apple
✔ Total gerado em 'Apple': 3450

📂 Categoria: strawberries
✔ Total gerado em 'strawberries': 3450

📂 Categoria: watermelon
✔ Total gerado em 'watermelon': 3450

📂 Categoria: kiwi
✔ Total gerado em 'kiwi': 3450

📂 Categoria: mango
✔ Total gerado em 'mango': 3465

📂 Categoria: Banana
✔ Total gerado em 'Banana': 3450

📂 Categoria: orange
✔ Total gerado em 'orange': 3450

🎉 PRONTO! TODAS AS VARIAÇÕES FORAM GERADAS.


## Treinar novamente

Como precisei aumentar a quantidade de imagens, o treino ficou mais pesado.
Este bloco ativa **memory growth** na GPU, permitindo que o TensorFlow utilize a memória de forma gradual, evitando erros de falta de VRAM.
O **MLflow** registra parâmetros, métricas e o modelo final, permitindo acompanhar e comparar experimentos facilmente.
Os datasets são criados e otimizados com `tf.data`, garantindo eficiência mesmo com um número maior de imagens.

In [5]:
import os

# -----------------------------------------------------------
# 🔥 DESATIVAR XLA (correto e realmente funcional)
# -----------------------------------------------------------
print("[XLA] Desativando completamente...")
os.environ["TF_XLA_FLAGS"] = "--tf_xla_enable_xla_devices=false"
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir=''"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
print("[XLA] OFF ✔️")

import tensorflow as tf
import mlflow
import mlflow.tensorflow

# -----------------------------------------------------------
# 🔥 GPU SAFE MODE (evita OOM)
# -----------------------------------------------------------
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
            print(f"[GPU] memory_growth ativado → {gpu}")
        except:
            pass
print("[GPU] Configurações aplicadas.\n")

# ---- MLflow autolog ----
mlflow.tensorflow.autolog()

train_dir = "/home/wallingson12/Github/Data science and ML/Classificador de frutas/train_variacoes"

# -----------------------------------------------------------
#  DETECTAR CLASSES
# -----------------------------------------------------------
image_size = (128, 128)

# 🔥 batch menor ajuda a evitar OOM
batch_size = 8

epochs = 15
validation_split = 0.2

class_names = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
num_classes = len(class_names)

print(f"Classes detectadas: {class_names}")
print(f"Número de classes: {num_classes}")

# -----------------------------------------------------------
# DATASETS
# -----------------------------------------------------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True,
    validation_split=validation_split,
    subset='training',
    seed=42
).prefetch(1)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False,
    validation_split=validation_split,
    subset='validation',
    seed=42
).prefetch(1)

# -----------------------------------------------------------
# MODELO
# -----------------------------------------------------------
model = build_model(image_size=image_size, num_classes=num_classes)

# -----------------------------------------------------------
# TREINO + MLflow
# -----------------------------------------------------------
with mlflow.start_run():

    mlflow.log_param("image_size", image_size)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("num_classes", num_classes)
    mlflow.log_param("validation_split", validation_split)

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )

    for epoch in range(len(history.history["loss"])):
        mlflow.log_metric("train_loss", history.history["loss"][epoch], epoch)
        mlflow.log_metric("train_accuracy", history.history["accuracy"][epoch], epoch)
        mlflow.log_metric("val_loss", history.history["val_loss"][epoch], epoch)
        mlflow.log_metric("val_accuracy", history.history["val_accuracy"][epoch], epoch)

    model.save("modelo_frutas.h5")
    mlflow.log_artifact("modelo_frutas.h5")

print("\n✅ Modelo salvo como modelo_frutas.h5")

[XLA] Desativando completamente...
[XLA] OFF ✔️
[GPU] memory_growth ativado → PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
[GPU] Configurações aplicadas.

Classes detectadas: ['Apple', 'Banana', 'avocado', 'cherry', 'kiwi', 'mango', 'orange', 'pinenapple', 'strawberries', 'watermelon']
Número de classes: 10
Found 34515 files belonging to 10 classes.
Using 27612 files for training.
Found 34515 files belonging to 10 classes.
Using 6903 files for validation.
Epoch 1/15


/home/wallingson12/.local/lib/python3.10/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
E0000 00:00:1764440230.055967    4624 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_1_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


3452/3452 ━━━━━━━━━━━━━━━━━━━━ 250s 71ms/step - accuracy: 0.4721 - loss: 1.7962 - val_accuracy: 0.6452 - val_loss: 1.3785
Epoch 2/15
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 247s 72ms/step - accuracy: 0.5951 - loss: 1.4446 - val_accuracy: 0.7463 - val_loss: 1.0028
Epoch 3/15
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 245s 71ms/step - accuracy: 0.6597 - loss: 1.2543 - val_accuracy: 0.8233 - val_loss: 0.8632
Epoch 4/15
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 255s 74ms/step - accuracy: 0.7110 - loss: 1.1145 - val_accuracy: 0.9410 - val_loss: 0.4875
Epoch 5/15
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 315s 91ms/step - accuracy: 0.7425 - loss: 1.0189 - val_accuracy: 0.9015 - val_loss: 0.5964
Epoch 6/15
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 313s 91ms/step - accuracy: 0.7698 - loss: 0.9393 - val_accuracy: 0.8770 - val_loss: 0.6445
Epoch 7/15
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 324s 94ms/step - accuracy: 0.7906 - loss: 0.8778 - val_accuracy: 0.8990 - val_loss: 0.5972
Epoch 8/15
3452/3452 ━━━━━━━━━━━━━━━━━━━━ 318s 92ms/step - accuracy: 0.8081 - lo

2025/11/29 16:29:41 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: Changing param values is not allowed. Param with key='batch_size' was already logged with value='8' for run ID='20245f9d94e040eb819b34b1c3d4ea16'. Attempted logging new value 'None'.



✅ Modelo salvo como modelo_frutas.h5


In [ ]:
## Acurácia de validação máxima: 0.4790 (93%)

## Visualização do Treinamento — Acurácia e Loss

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    """
    Plota acurácia e loss do treinamento a partir do objeto History do Keras.
    """
    # Acurácia
    plt.figure(figsize=(8,5))
    plt.plot(history.history['accuracy'], marker='o')
    plt.plot(history.history['val_accuracy'], marker='o')
    plt.title('Treino x Validação - Acurácia')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.grid(True)
    plt.legend(['Treino', 'Validação'])
    plt.show()

    # Loss
    plt.figure(figsize=(8,5))
    plt.plot(history.history['loss'], marker='o')
    plt.plot(history.history['val_loss'], marker='o')
    plt.title('Treino x Validação - Loss')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend(['Treino', 'Validação'])
    plt.show()

plot_training_history(history)

Essa lista mostra **apenas a precisão de cada classe**, ou seja, a proporção de previsões corretas entre todas as previsões feitas para aquela classe específica.

Alguns pontos importantes:

- Classes como **kiwi, watermelon e pinenapple** têm precisão muito alta (>0.85), o que indica que o modelo raramente erra quando prediz essas frutas.
- Classes como **avocado e banana** têm precisão muito baixa (<0.05), mostrando que o modelo frequentemente confunde essas frutas com outras.
- O modelo parece **tender a prever certas classes com mais confiança** e confundir outras, possivelmente por ter **menos exemplos de treino ou mais similaridade visual** entre frutas.
- Isso ajuda a identificar **onde melhorar o dataset ou ajustar o modelo**, por exemplo: aumentar imagens de avocado/banana ou aplicar técnicas de balanceamento.

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Matriz de confusão
cm = confusion_matrix(true_labels, pred_labels)

# Precisão por classe = diagonal / soma da coluna
precision_per_class = np.diag(cm) / cm.sum(axis=0)

# Print formatado como tabela
print("Precisão por Classe:\n")
for cls, prec in zip(class_names, precision_per_class):
    print(f"{cls:12s} : {prec:.2f}")